In [1]:
import torch
import torch.nn as nn
import gym
import numpy as np
import torch.optim as optim
import random
import time


In [2]:
class Model(nn.Module):
    def __init__(self, observation, action):
        super(Model, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(observation, 32, bias=False),
            nn.Linear(32, 100, bias=False),
            nn.Linear(100,32, bias =False),
            nn.Linear(32, action, bias = False),
        )
        
    def forward(self, input):
        return self.network(input)

In [3]:
done = False
learning_rate = 0.01
discount = 1
n_episodes = 25000
epsilon = 0.9
# memory_d = 0
# final_d = 8
episode_rewards = np.array([])
avg_loss = 0
rewards_path = 'rewards.npy'

env = gym.make("CartPole-v1")
env.reset()
device = torch.device("cuda") 
# Make 2 models
q_hat = Model(env.observation_space.shape[0], env.action_space.n).to(device)
# q_hat_target = Model(env.observation_space.shape[0], env.action_space.n).to(device)

In [4]:
#Optimizer

criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(q_hat.parameters(), lr = 1e-5)
        

In [5]:
# env.close()
print(env.reset()) #[cart_position, cart_velocity, angle, angular velocity]

[-0.01460769  0.00821473 -0.02054977  0.02625358]


In [6]:
for episode in range(n_episodes):
    
    state = torch.from_numpy(env.reset()).float().to(device)
    done = False
    episode_reward = 0
    total_loss = 0
    
    if episode%300 == 0:

        RENDER = True
    else:
        RENDER = False
        
    while not done:
        
        if RENDER:
            time.sleep(0.05)
            env.render()
            
#         memory_d += 1
#         if memory_d%final_d == 0:
#             q_hat_target.load_state_dict(q_hat.state_dict()) # storing q_hat into q_hat_target for memory
#             memory_d = 0
        
        q = q_hat(state)
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = torch.argmax(q).item()
            
        next_state, reward, done, _ = env.step(action)
        next_state = torch.from_numpy(next_state).float().to(device)
        
        q_target = q.clone()
        q_target[action] = torch.tensor(reward) + discount*torch.max(q_hat(next_state))-q[action]
        
        if done:
            q_target[action]=torch.tensor(reward)
            
            
        #calculate loss
        loss = criterion(q,q_target.detach())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        state = next_state
        
        episode_reward += reward
        total_loss += loss.item()
        
    epsilon *= 0.9999
    episode_rewards = np.append(episode_rewards, episode_reward)
    avg_loss += total_loss
    
    env.close()
    if episode%100 == 0:
        print('Episode: %5d | Average Loss: %5.2f | Epsilon: %2.4f | Avg. Reward: %5.4f'%(episode, avg_loss/100, epsilon, np.mean(episode_rewards[-100:])))
        avg_loss = 0
        ep_reward = 0
        np.save(rewards_path, episode_rewards)

Episode:     0 | Average Loss:  0.00 | Epsilon: 0.8999 | Avg. Reward: 11.0000
Episode:   100 | Average Loss:  0.50 | Epsilon: 0.8910 | Avg. Reward: 27.1100
Episode:   200 | Average Loss:  0.50 | Epsilon: 0.8821 | Avg. Reward: 28.8600
Episode:   300 | Average Loss:  0.49 | Epsilon: 0.8733 | Avg. Reward: 28.8000
Episode:   400 | Average Loss:  0.50 | Epsilon: 0.8646 | Avg. Reward: 29.3400
Episode:   500 | Average Loss:  0.50 | Epsilon: 0.8560 | Avg. Reward: 31.3400
Episode:   600 | Average Loss:  0.49 | Epsilon: 0.8475 | Avg. Reward: 29.5500
Episode:   700 | Average Loss:  0.51 | Epsilon: 0.8391 | Avg. Reward: 32.3100
Episode:   800 | Average Loss:  0.49 | Epsilon: 0.8307 | Avg. Reward: 29.1800
Episode:   900 | Average Loss:  0.50 | Epsilon: 0.8225 | Avg. Reward: 34.7100
Episode:  1000 | Average Loss:  0.51 | Epsilon: 0.8143 | Avg. Reward: 33.2800
Episode:  1100 | Average Loss:  0.50 | Epsilon: 0.8062 | Avg. Reward: 31.5800
Episode:  1200 | Average Loss:  0.50 | Epsilon: 0.7981 | Avg. Re

KeyboardInterrupt: 